# Feature table calculation in action
## load data

In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time

import squidpy as sp

import skimage.feature as sk_image

# path to "raw" dataset folder
# BASE_PATH = "../../data"
BASE_PATH = "/storage/groups/ml01/datasets/raw/20200909_PublicVisium_giovanni.palla"
dataset_name = "V1_Adult_Mouse_Brain"
dataset_folder = os.path.join(
    BASE_PATH, "20191205_10XVisium_MouseBrainCoronal_giovanni.palla"
)
image_file = os.path.join(dataset_folder, f"{dataset_name}_image.tif")

In [2]:
# read the corresponding adata object
adata = sc.read_visium(
    dataset_folder, count_file=f"{dataset_name}_filtered_feature_bc_matrix.h5"
)
img = sp.im.ImageContainer(image_file)

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
/home/marius.lange/miniconda3/envs/squidpy/lib/python3.8/site-packages/rasterio/__init__.py:221: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


## Example of calculating the feature table
The available feature in the current version are: "hog", "texture", "summary", "color_hist"


In [3]:
features = features=["hog", "texture", "summary", "color_hist"]

### write table in adata.obsm

In [4]:
sp.im.calculate_image_features(adata, img, ['summary'], key='features')


In [5]:
adata.obsm['features']

,summary_quantile_0.9_ch_0,summary_quantile_0.5_ch_0,summary_quantile_0.1_ch_0,summary_quantile_0.9_ch_1,summary_quantile_0.5_ch_1,summary_quantile_0.1_ch_1,summary_quantile_0.9_ch_2,summary_quantile_0.5_ch_2,summary_quantile_0.1_ch_2
AAACAAGTATCTCCCA-1,132.0,111.0,77.0,102.0,79.0,54.0,138.0,111.0,88.0
AAACAATCTACTAGCA-1,140.0,111.0,80.0,87.0,60.0,39.0,135.0,108.0,84.0
AAACACCAATAACTGC-1,132.0,116.0,89.0,109.0,91.0,67.0,130.0,117.0,97.0
AAACAGAGCGACTCCT-1,136.0,116.0,93.0,115.0,82.0,58.0,137.0,113.0,89.0
AAACCGGGTAGGTACC-1,137.0,112.0,83.0,103.0,78.0,54.0,133.0,113.0,92.0
...,...,...,...,...,...,...,...,...,...
TTGTTGTGTGTCAAGA-1,135.0,107.0,45.0,104.0,74.0,33.0,143.0,111.0,74.0
TTGTTTCACATCCAGG-1,139.0,107.0,64.0,112.0,81.0,49.0,140.0,113.0,84.0
TTGTTTCATTAGTCTA-1,135.0,111.0,73.0,107.0,81.0,54.0,134.0,114.0,89.0
TTGTTTCCATACAACT-1,136.0,110.0,77.0,99.0,73.0,49.0,133.0,112.0,88.0


### return features table

In [6]:
feature_table = sp.im.calculate_image_features(adata[0:10], img, ['summary'])
feature_table

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


### change size and scale of crops

In [7]:
feature_table = sp.im.calculate_image_features(adata[0:10], img, ['summary'], size=2, scale=0.5)
feature_table

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


# Parallelization performance

## Single core

In [8]:
%%timeit
sp.im.calculate_image_features(adata, img, ['summary'], key='features',
                               n_jobs=1, show_progress_bar=False)

13.2 s ± 18 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## 8 cores

In [9]:
%%timeit
sp.im.calculate_image_features(adata, img, ['summary'], key='features',
                               n_jobs=8, show_progress_bar=False)

3.13 s ± 61 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
